In [1]:
import json
from gensim.models import Word2Vec
import re
import csv
from nltk.tokenize import word_tokenize
import nltk
from gensim.models import KeyedVectors

C:\Users\PC-Axel\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
def preprocess(article):
    article = article.encode('utf-8')
    article = str(article.lower())
    return removeTagsInterpuntion(article)

def removeTagsInterpuntion(article): #remove URL's, HTML-tags and interpuntion
    article = re.sub("<\w*>", '', article)
    article = re.sub("<\w*\s\/>", '', article)
    article = re.sub("^https?:\/\/.*[\r\n]*", '', article)
    article = re.sub('\\\\x\S.', '', article)
    article = re.sub('[^a-z\s]', '', article)
    article = re.sub('\d', '',article)
    return article

def json_readr(file):
    for line in open(file, mode="r"):
        yield json.loads(line)

def loadData(path, texts, labels, urls, highLevelLabels = True):
    with open(path) as data_file:    
        for line in data_file:
            jsons = re.split('\]\[', line)
            data = []
            for newline in jsons:
                newline = re.sub('\]\[', '\[', newline)
                if newline[0] is not "[":
                    newline = "[" + newline
                if newline[len(newline)-1] is not "}" and newline[len(newline)-1] is not "]":
                    newline+= "}"
                if newline[len(newline)-1] is not "]":
                    newline+= "]"
                betterload = json.loads(newline)
                data.extend(betterload)
    for article in data:
        text = preprocess(article["content"])
        if len(article["category"]) != 0:
            if len(text.split()) > 9 and article["category"][0] != "NOCAT": #Remove small sentences
                if article["url"] not in urls and "kv-tk" not in article["url"]:
                    urls.append(article["url"])
                    texts.append(word_tokenize(text))
                    if highLevelLabels:
                        label = []
                        for lowLevelLabel in article["category"]:
                            if lowLevelLabel.split("|")[0] not in label:
                                label.append(lowLevelLabel.split("|")[0])
                        labels.append(label)
                    else:
                        labels.append(article["category"])
    return texts, labels, urls

In [3]:
texts = []
labels = []
urls = []
paths = [r"C:\Users\PC-Axel\Documents\github\thesis\Data\PoliFLW Data\kamerstukken_goed_jan2016.json",
        r"C:\Users\PC-Axel\Documents\github\thesis\Data\PoliFLW Data\kamerstukken_goed_jan2015.json",
        r"C:\Users\PC-Axel\Documents\github\thesis\Data\PoliFLW Data\kamerstukken_goed_okt2015.json",
        r"C:\Users\PC-Axel\Documents\github\thesis\Data\PoliFLW Data\kamerstukken_goed_2017.json",
        r"C:\Users\PC-Axel\Documents\github\thesis\Data\PoliFLW Data\kamerstukken_goed_2010-2014.json",
        r"C:\Users\PC-Axel\Documents\github\thesis\Data\PoliFLW Data\kamerstukken_goed_2001-2003.json",
        r"C:\Users\PC-Axel\Documents\github\thesis\Data\PoliFLW Data\kamerstukken_goed_2004-2006.json",
        r"C:\Users\PC-Axel\Documents\github\thesis\Data\PoliFLW Data\kamerstukken_goed_2007-2009.json"]

for path in paths:
    texts, labels, urls = loadData(path, texts, labels, urls)
    print("path done")

path done
path done
path done
path done
path done
path done
path done
path done


In [4]:
with open(r"C:\Users\PC-Axel\Documents\GitHub\thesis\Code\Data preparation\alleGemeentes.csv", 'r') as csvfile:
    next(csvfile)
    spamreader = csv.reader(csvfile, delimiter='|')    
    for row in spamreader:
         texts.append(word_tokenize(preprocess(row[1])))
            
with open(r"C:\Users\PC-Axel\Documents\GitHub\thesis\Code\Data preparation\alleGemeentes2.csv", 'r') as csvfile:
    next(csvfile)
    spamreader = csv.reader(csvfile, delimiter='|')
    for row in spamreader:
         texts.append(word_tokenize(preprocess(row[1])))

In [5]:
model = Word2Vec(texts, size=160, window=5, min_count=5)

In [32]:
word_vectors2 = KeyedVectors.load_word2vec_format(r"C:\Users\PC-Axel\Documents\Codeer projecten\Word2Vec Vectoren\Nederlandse word2vec\combined-160.txt", binary=False)

In [7]:
word_vectors = model.wv

In [9]:
print(word_vectors.most_similar_cosmul(positive=['buiten']))
#print(word_vectors2.most_similar_cosmul(positive=['belasting']))

[('buitenr', 0.8683865666389465), ('binnen', 0.7571752071380615), ('daarbuiten', 0.7403473854064941), ('elders', 0.7342738509178162), ('boven', 0.7157936692237854), ('vrij', 0.697746992111206), ('binnenr', 0.6930598020553589), ('beneden', 0.6908529996871948), ('omgekeerd', 0.6868816018104553), ('daaronder', 0.6829747557640076)]


In [10]:
word_vectors.save_word2vec_format("zelfgemaakte-w2v.txt")